# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources in the README.md file
- Happy learning!

In [1]:
# Import your libraries:

%matplotlib inline

import numpy as np
import pandas as pd
from datetime import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import re

In this lab, we will explore a dataset that describes websites with different features and labels them either benign or malicious . We will use supervised learning algorithms to figure out what feature patterns malicious websites are likely to have and use our model to predict malicious websites.

# Challenge 1 - Explore The Dataset

Let's start by exploring the dataset. First load the data file:

In [2]:
websites = pd.read_csv('../website.csv')

#### Explore the data from an bird's-eye view.

You should already been very familiar with the procedures now so we won't provide the instructions step by step. Reflect on what you did in the previous labs and explore the dataset.

Things you'll be looking for:

* What the dataset looks like?
* What are the data types?
* Which columns contain the features of the websites?
* Which column contains the feature we will predict? What is the code standing for benign vs malicious websites?
* Do we need to transform any of the columns from categorical to ordinal values? If so what are these columns?

Feel free to add additional cells for your explorations. Make sure to comment what you find out.

In [3]:
websites.head()

URL  URL_LENGTH  NUMBER_SPECIAL_CHARACTERS     CHARSET  \
0   M0_109          16                          7  iso-8859-1   
1  B0_2314          16                          6       UTF-8   
2   B0_911          16                          6    us-ascii   
3   B0_113          17                          6  ISO-8859-1   
4   B0_403          17                          6       UTF-8   

                  SERVER  CONTENT_LENGTH WHOIS_COUNTRY WHOIS_STATEPRO  \
0                  nginx           263.0          None           None   
1          Apache/2.4.10         15087.0          None           None   
2  Microsoft-HTTPAPI/2.0           324.0          None           None   
3                  nginx           162.0            US             AK   
4                   None        124140.0            US             TX   

      WHOIS_REGDATE WHOIS_UPDATED_DATE  ...  DIST_REMOTE_TCP_PORT  REMOTE_IPS  \
0  10/10/2015 18:21               None  ...                     0           2   
1              None               None  ...                     7           4   
2              None               None  ...                     0           0   
3    7/10/1997 4:00    12/09/2013 0:45  ...                    22           3   
4   12/05/1996 0:00    11/04/2017 0:00  ...                     2           5   

   APP_BYTES  SOURCE_APP_PACKETS  REMOTE_APP_PACKETS  SOURCE_APP_BYTES  \
0        700                   9                  10              1153   
1       1230                  17                  19              1265   
2          0                   0                   0                 0   
3       3812                  39                  37             18784   
4       4278                  61                  62            129889   

   REMOTE_APP_BYTES  APP_PACKETS  DNS_QUERY_TIMES  Type  
0               832            9              2.0     1  
1              1230           17              0.0     0  
2                 0            0              0.0     0  
3              4380           39              8.0     0  
4              4586           61              4.0     0  

[5 rows x 21 columns]

In [4]:
websites.dtypes

URL                           object
URL_LENGTH                     int64
NUMBER_SPECIAL_CHARACTERS      int64
CHARSET                       object
SERVER                        object
CONTENT_LENGTH               float64
WHOIS_COUNTRY                 object
WHOIS_STATEPRO                object
WHOIS_REGDATE                 object
WHOIS_UPDATED_DATE            object
TCP_CONVERSATION_EXCHANGE      int64
DIST_REMOTE_TCP_PORT           int64
REMOTE_IPS                     int64
APP_BYTES                      int64
SOURCE_APP_PACKETS             int64
REMOTE_APP_PACKETS             int64
SOURCE_APP_BYTES               int64
REMOTE_APP_BYTES               int64
APP_PACKETS                    int64
DNS_QUERY_TIMES              float64
Type                           int64
dtype: object

In [5]:
websites.describe()

URL_LENGTH  NUMBER_SPECIAL_CHARACTERS  CONTENT_LENGTH  \
count  1781.000000                1781.000000      969.000000   
mean     56.961258                  11.111735    11726.927761   
std      27.555586                   4.549896    36391.809051   
min      16.000000                   5.000000        0.000000   
25%      39.000000                   8.000000      324.000000   
50%      49.000000                  10.000000     1853.000000   
75%      68.000000                  13.000000    11323.000000   
max     249.000000                  43.000000   649263.000000   

       TCP_CONVERSATION_EXCHANGE  DIST_REMOTE_TCP_PORT   REMOTE_IPS  \
count                1781.000000           1781.000000  1781.000000   
mean                   16.261089              5.472768     3.060640   
std                    40.500975             21.807327     3.386975   
min                     0.000000              0.000000     0.000000   
25%                     0.000000              0.000000     0.000000   
50%                     7.000000              0.000000     2.000000   
75%                    22.000000              5.000000     5.000000   
max                  1194.000000            708.000000    17.000000   

          APP_BYTES  SOURCE_APP_PACKETS  REMOTE_APP_PACKETS  SOURCE_APP_BYTES  \
count  1.781000e+03         1781.000000         1781.000000      1.781000e+03   
mean   2.982339e+03           18.540146           18.746210      1.589255e+04   
std    5.605057e+04           41.627173           46.397969      6.986193e+04   
min    0.000000e+00            0.000000            0.000000      0.000000e+00   
25%    0.000000e+00            0.000000            0.000000      0.000000e+00   
50%    6.720000e+02            8.000000            9.000000      5.790000e+02   
75%    2.328000e+03           26.000000           25.000000      9.806000e+03   
max    2.362906e+06         1198.000000         1284.000000      2.060012e+06   

       REMOTE_APP_BYTES  APP_PACKETS  DNS_QUERY_TIMES         Type  
count      1.781000e+03  1781.000000      1780.000000  1781.000000  
mean       3.155599e+03    18.540146         2.263483     0.121280  
std        5.605378e+04    41.627173         2.930853     0.326544  
min        0.000000e+00     0.000000         0.000000     0.000000  
25%        0.000000e+00     0.000000         0.000000     0.000000  
50%        7.350000e+02     8.000000         0.000000     0.000000  
75%        2.701000e+03    26.000000         4.000000     0.000000  
max        2.362906e+06  1198.000000        20.000000     1.000000

#### Next, evaluate if the columns in this dataset are strongly correlated.

In the Mushroom supervised learning lab we did recently, we mentioned we are concerned if our dataset has strongly correlated columns because if it is the case we need to choose certain ML algorithms instead of others. We need to evaluate this for our dataset now.

Luckily, most of the columns in this dataset are ordinal which makes things a lot easier for us. In the next cells below, evaluate the level of collinearity of the data.

We provide some general directions for you to consult in order to complete this step:

1. You will create a correlation matrix using the numeric columns in the dataset.

1. Create a heatmap using `seaborn` to visualize which columns have high collinearity.

1. Comment on which columns you might need to remove due to high collinearity.

In [6]:
corr=websites.corr()

In [7]:
def correlation_matrix(corr, title=""):
    mask = np.zeros_like(corr, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True

    plt.subplots(figsize=(15, 10))
    cmap = sns.diverging_palette(6, 255, as_cmap=True)
    
    chart = sns.heatmap(corr, mask=mask, cmap=cmap, center=0, linewidths=.5, annot=True, fmt='.2f')
    plt.title(title, fontsize=16)
    
    return chart

In [8]:
corr_plot = correlation_matrix(corr, title='Correlação entre as variáveis disponíveis')

# Challenge 2 - Remove Column Collinearity.

From the heatmap you created, you should have seen at least 3 columns that can be removed due to high collinearity. Remove these columns from the dataset.

Note that you should remove as few columns as you can. You don't have to remove all the columns at once. But instead, try removing one column, then produce the heatmap again to determine if additional columns should be removed. As long as the dataset no longer contains columns that are correlated for over 90%, you can stop. Also, keep in mind when two columns have high collinearity, you only need to remove one of them but not both.

In the cells below, remove as few columns as you can to eliminate the high collinearity in the dataset. Make sure to comment on your way so that the instructional team can learn about your thinking process which allows them to give feedback. At the end, print the heatmap again.

In [9]:
websites.columns

Index(['URL', 'URL_LENGTH', 'NUMBER_SPECIAL_CHARACTERS', 'CHARSET', 'SERVER',
       'CONTENT_LENGTH', 'WHOIS_COUNTRY', 'WHOIS_STATEPRO', 'WHOIS_REGDATE',
       'WHOIS_UPDATED_DATE', 'TCP_CONVERSATION_EXCHANGE',
       'DIST_REMOTE_TCP_PORT', 'REMOTE_IPS', 'APP_BYTES', 'SOURCE_APP_PACKETS',
       'REMOTE_APP_PACKETS', 'SOURCE_APP_BYTES', 'REMOTE_APP_BYTES',
       'APP_PACKETS', 'DNS_QUERY_TIMES', 'Type'],
      dtype='object')

In [10]:
websites.drop(['TCP_CONVERSATION_EXCHANGE'], axis = 1, inplace = True)

In [11]:
corr2=websites.corr()

In [12]:
corr_plot = correlation_matrix(corr2, title='Correlação entre as variáveis disponíveis')

In [13]:
websites.drop(['SOURCE_APP_PACKETS'], axis = 1, inplace = True)

In [14]:
corr3=websites.corr()

In [15]:
corr_plot = correlation_matrix(corr3, title='Correlação entre as variáveis disponíveis')

In [16]:
websites.drop(['REMOTE_APP_PACKETS'], axis = 1, inplace = True)

In [17]:
corr4=websites.corr()

In [18]:
corr_plot = correlation_matrix(corr4, title='Correlação entre as variáveis disponíveis')

# Challenge 3 - Handle Missing Values

The next step would be handling missing values. **We start by examining the number of missing values in each column, which you will do in the next cell.**

In [19]:
websites.isnull().sum()

URL                            0
URL_LENGTH                     0
NUMBER_SPECIAL_CHARACTERS      0
CHARSET                        0
SERVER                         1
CONTENT_LENGTH               812
WHOIS_COUNTRY                  0
WHOIS_STATEPRO                 0
WHOIS_REGDATE                  0
WHOIS_UPDATED_DATE             0
DIST_REMOTE_TCP_PORT           0
REMOTE_IPS                     0
APP_BYTES                      0
SOURCE_APP_BYTES               0
REMOTE_APP_BYTES               0
APP_PACKETS                    0
DNS_QUERY_TIMES                1
Type                           0
dtype: int64

If you remember in the previous labs, we drop a column if the column contains a high proportion of missing values. After dropping those problematic columns, we drop the rows with missing values.

#### In the cells below, handle the missing values from the dataset. Remember to comment the rationale of your decisions.

In [20]:
websites.shape

(1781, 18)

In [21]:
websites['CONTENT_LENGTH'].isnull().sum()/len(websites)

0.45592363840539024

In [22]:
websites.drop(['CONTENT_LENGTH'], axis = 1, inplace = True)

In [23]:
mask= websites['SERVER'].isnull() == True
websites[mask]

URL  URL_LENGTH  NUMBER_SPECIAL_CHARACTERS CHARSET SERVER  \
1306  B0_2134          66                         11   UTF-8    NaN   

     WHOIS_COUNTRY        WHOIS_STATEPRO    WHOIS_REGDATE WHOIS_UPDATED_DATE  \
1306            US  District of Columbia  1/02/2003 16:44    2/02/2017 17:11   

      DIST_REMOTE_TCP_PORT  REMOTE_IPS  APP_BYTES  SOURCE_APP_BYTES  \
1306                     2           4       1026               246   

      REMOTE_APP_BYTES  APP_PACKETS  DNS_QUERY_TIMES  Type  
1306              1026           15              0.0     0

In [24]:
websites.drop([1306], axis = 0, inplace = True)

In [25]:
mask2= websites['DNS_QUERY_TIMES'].isnull() == True
websites[mask2]

URL  URL_LENGTH  NUMBER_SPECIAL_CHARACTERS CHARSET  SERVER  \
1659  B0_772         101                         15   UTF-8  Apache   

     WHOIS_COUNTRY WHOIS_STATEPRO    WHOIS_REGDATE WHOIS_UPDATED_DATE  \
1659            CA             ON  14/12/2009 0:00    13/12/2016 0:00   

      DIST_REMOTE_TCP_PORT  REMOTE_IPS  APP_BYTES  SOURCE_APP_BYTES  \
1659                     2           2        276               416   

      REMOTE_APP_BYTES  APP_PACKETS  DNS_QUERY_TIMES  Type  
1659               276            4              NaN     0

In [26]:
websites.drop([1659], axis = 0, inplace = True)

#### Again, examine the number of missing values in each column. 

If all cleaned, proceed. Otherwise, go back and do more cleaning.

In [27]:
websites.isnull().sum()

URL                          0
URL_LENGTH                   0
NUMBER_SPECIAL_CHARACTERS    0
CHARSET                      0
SERVER                       0
WHOIS_COUNTRY                0
WHOIS_STATEPRO               0
WHOIS_REGDATE                0
WHOIS_UPDATED_DATE           0
DIST_REMOTE_TCP_PORT         0
REMOTE_IPS                   0
APP_BYTES                    0
SOURCE_APP_BYTES             0
REMOTE_APP_BYTES             0
APP_PACKETS                  0
DNS_QUERY_TIMES              0
Type                         0
dtype: int64

# Challenge 4 - Handle `WHOIS_*` Categorical Data

There are several categorical columns we need to handle. These columns are:

* `URL`
* `CHARSET`
* `SERVER`
* `WHOIS_COUNTRY`
* `WHOIS_STATEPRO`
* `WHOIS_REGDATE`
* `WHOIS_UPDATED_DATE`

How to handle string columns is always case by case. Let's start by working on `WHOIS_COUNTRY`. Your steps are:

1. List out the unique values of `WHOIS_COUNTRY`.
1. Consolidate the country values with consistent country codes. For example, the following values refer to the same country and should use consistent country code:
    * `CY` and `Cyprus`
    * `US` and `us`
    * `SE` and `se`
    * `GB`, `United Kingdom`, and `[u'GB'; u'UK']`

#### In the cells below, fix the country values as intructed above.

In [28]:
websites['WHOIS_COUNTRY'].unique()

array(['None', 'US', 'SC', 'GB', 'UK', 'RU', 'AU', 'CA', 'PA', 'se', 'IN',
       'LU', 'TH', "[u'GB'; u'UK']", 'FR', 'NL', 'UG', 'JP', 'CN', 'SE',
       'SI', 'IL', 'ru', 'KY', 'AT', 'CZ', 'PH', 'BE', 'NO', 'TR', 'LV',
       'DE', 'ES', 'BR', 'us', 'KR', 'HK', 'UA', 'CH', 'United Kingdom',
       'BS', 'PK', 'IT', 'Cyprus', 'BY', 'AE', 'IE', 'UY', 'KG'],
      dtype=object)

In [29]:
websites['WHOIS_COUNTRY'] = websites['WHOIS_COUNTRY'].str.upper()

In [30]:
def country_UK (country):
    a = re.sub("UNITED KINGDOM","UK",country)
    return a

In [31]:
websites['WHOIS_COUNTRY'] = websites['WHOIS_COUNTRY'].apply(country_UK)

In [32]:
def country_CY (country):
    a = re.sub("CYPRUS","CY",country)
    return a

In [33]:
websites['WHOIS_COUNTRY'] = websites['WHOIS_COUNTRY'].apply(country_CY)

In [34]:
def country_list (country):
    a = country.replace(r"[U'GB'; U'UK']","GB")
    return a

In [35]:
websites['WHOIS_COUNTRY'] = websites['WHOIS_COUNTRY'].apply(country_list)

In [36]:
websites['WHOIS_COUNTRY'].unique()

array(['NONE', 'US', 'SC', 'GB', 'UK', 'RU', 'AU', 'CA', 'PA', 'SE', 'IN',
       'LU', 'TH', 'FR', 'NL', 'UG', 'JP', 'CN', 'SI', 'IL', 'KY', 'AT',
       'CZ', 'PH', 'BE', 'NO', 'TR', 'LV', 'DE', 'ES', 'BR', 'KR', 'HK',
       'UA', 'CH', 'BS', 'PK', 'IT', 'CY', 'BY', 'AE', 'IE', 'UY', 'KG'],
      dtype=object)

In [37]:
# Your code here


In [38]:
def fernanda_prigs(col_name, value2find, value2rep):
    websites.loc[websites[col_name]==value2find, col_name] = value2rep

#### After verifying, now let's keep the top 10 values of the column and re-label other columns with `OTHER`.

In [39]:
websites['WHOIS_COUNTRY'].value_counts().nlargest(10)

US      1105
NONE     306
CA        83
ES        63
AU        35
GB        24
PA        21
UK        11
JP        11
IN        10
Name: WHOIS_COUNTRY, dtype: int64

In [40]:
top10_countries = websites['WHOIS_COUNTRY'].value_counts().nlargest(10)

In [41]:
for country in websites['WHOIS_COUNTRY']:
    if country not in top10_countries:
        fernanda_prigs('WHOIS_COUNTRY', country, 'OTHER')

In [42]:
websites['WHOIS_COUNTRY'].value_counts()

US       1105
NONE      306
OTHER     110
CA         83
ES         63
AU         35
GB         24
PA         21
JP         11
UK         11
IN         10
Name: WHOIS_COUNTRY, dtype: int64

Now since `WHOIS_COUNTRY` has been re-labelled, we don't need `WHOIS_STATEPRO` any more because the values of the states or provinces may not be relevant any more. We'll drop this column.

In addition, we will also drop `WHOIS_REGDATE` and `WHOIS_UPDATED_DATE`. These are the registration and update dates of the website domains. Not of our concerns.

#### In the next cell, drop `['WHOIS_STATEPRO', 'WHOIS_REGDATE', 'WHOIS_UPDATED_DATE']`.

In [43]:
websites.drop(['WHOIS_STATEPRO'], axis=1, inplace=True)
websites.drop(['WHOIS_REGDATE'], axis=1, inplace=True)
websites.drop(['WHOIS_UPDATED_DATE'], axis=1, inplace=True)

In [44]:
websites.columns

Index(['URL', 'URL_LENGTH', 'NUMBER_SPECIAL_CHARACTERS', 'CHARSET', 'SERVER',
       'WHOIS_COUNTRY', 'DIST_REMOTE_TCP_PORT', 'REMOTE_IPS', 'APP_BYTES',
       'SOURCE_APP_BYTES', 'REMOTE_APP_BYTES', 'APP_PACKETS',
       'DNS_QUERY_TIMES', 'Type'],
      dtype='object')

# Challenge 5 - Handle Remaining Categorical Data & Convert to Ordinal

Now print the `dtypes` of the data again. Besides `WHOIS_COUNTRY` which we already fixed, there should be 3 categorical columns left: `URL`, `CHARSET`, and `SERVER`.

In [45]:
websites.dtypes

URL                           object
URL_LENGTH                     int64
NUMBER_SPECIAL_CHARACTERS      int64
CHARSET                       object
SERVER                        object
WHOIS_COUNTRY                 object
DIST_REMOTE_TCP_PORT           int64
REMOTE_IPS                     int64
APP_BYTES                      int64
SOURCE_APP_BYTES               int64
REMOTE_APP_BYTES               int64
APP_PACKETS                    int64
DNS_QUERY_TIMES              float64
Type                           int64
dtype: object

#### `URL` is easy. We'll simply drop it because it has too many unique values that there's no way for us to consolidate.

In [46]:
websites.drop(['URL'], axis=1, inplace=True)

#### Print the unique value counts of `CHARSET`. You see there are only a few unique values. So we can keep it as it is.

In [47]:
websites['CHARSET'].value_counts()

UTF-8           674
ISO-8859-1      427
utf-8           379
us-ascii        155
iso-8859-1      134
None              7
windows-1252      1
ISO-8859          1
windows-1251      1
Name: CHARSET, dtype: int64

`SERVER` is a little more complicated. Print its unique values and think about how you can consolidate those values.

#### Before you think of your own solution, don't read the instructions that come next.

In [48]:
websites['SERVER'].value_counts()

Apache                                                                                                         385
nginx                                                                                                          211
None                                                                                                           175
Microsoft-HTTPAPI/2.0                                                                                          113
cloudflare-nginx                                                                                                94
Microsoft-IIS/7.5                                                                                               51
Server                                                                                                          49
GSE                                                                                                             49
YouTubeFrontEnd                                                                 

![Think Hard](../think-hard.jpg)

Although there are so many unique values in the `SERVER` column, there are actually only 3 main server types: `Microsoft`, `Apache`, and `nginx`. Just check if each `SERVER` value contains any of those server types and re-label them. For `SERVER` values that don't contain any of those substrings, label with `Other`.

At the end, your `SERVER` column should only contain 4 unique values: `Microsoft`, `Apache`, `nginx`, and `Other`.

In [49]:
def word_in_server(df, col_name, value2find, value2rep):
    df.loc[(df[col_name].str.contains(value2find)) & (df[col_name].notnull()), col_name] = value2rep
    df.loc[(df[col_name].str.contains(value2find)) & (df[col_name].notnull()), "flag"] = 1
    
values = ["Microsoft","Apache","nginx"]

for value in values:
    word_in_server(websites, "SERVER", value, value)

In [54]:
websites.loc[websites["flag"].isna(), "SERVER"] = "Other"

def word_in_server(col_name, value2find, value2rep):
    a = re.sub(f'\w*{value2find}.*', value2rep, websites[col_name])
    return a

def word_in_server_microsoft(col_name):
    a = re.sub('\w*Microsoft.*', 'Microsoft',col_name)
    return a

websites['SERVER'] = websites['SERVER'].apply(word_in_server_microsoft)

def word_in_server_apache(col_name):
    a = re.sub('\w*Apache.*', 'Apache',col_name)
    return a

websites['SERVER'] = websites['SERVER'].apply(word_in_server_apache)

def word_in_server_nginx(col_name):
    a = re.sub('\w*nginx.*', 'nginx',col_name)
    return a

websites['SERVER'] = websites['SERVER'].apply(word_in_server_nginx)

lst_server = ['Apache', 'nginx', 'Microsoft']
for server in websites['SERVER']:
    if server not in lst_server:
        websites.loc[websites['SERVER']==server] = 'Other'

In [55]:
websites['SERVER'].value_counts()

Apache       642
Other        504
nginx        435
Microsoft    198
Name: SERVER, dtype: int64

In [57]:
websites.drop(['flag'], axis=1, inplace = True)

OK, all our categorical data are fixed now. **Let's convert them to ordinal data using Pandas' `get_dummies` function ([documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)).** Make sure you drop the categorical columns by passing `drop_first=True` to `get_dummies` as we don't need them any more. **Also, assign the data with dummy values to a new variable `website_dummy`.**

In [59]:
website_dummy = pd.get_dummies(websites, drop_first=True)

Now, inspect `website_dummy` to make sure the data and types are intended - there shouldn't be any categorical columns at this point.

In [61]:
website_dummy.head()

URL_LENGTH  NUMBER_SPECIAL_CHARACTERS  DIST_REMOTE_TCP_PORT  REMOTE_IPS  \
0          16                          7                     0           2   
1          16                          6                     7           4   
2          16                          6                     0           0   
3          17                          6                    22           3   
4          17                          6                     2           5   

   APP_BYTES  SOURCE_APP_BYTES  REMOTE_APP_BYTES  APP_PACKETS  \
0        700              1153               832            9   
1       1230              1265              1230           17   
2          0                 0                 0            0   
3       3812             18784              4380           39   
4       4278            129889              4586           61   

   DNS_QUERY_TIMES  Type  ...  WHOIS_COUNTRY_CA  WHOIS_COUNTRY_ES  \
0              2.0     1  ...                 0                 0   
1              0.0     0  ...                 0                 0   
2              0.0     0  ...                 0                 0   
3              8.0     0  ...                 0                 0   
4              4.0     0  ...                 0                 0   

   WHOIS_COUNTRY_GB  WHOIS_COUNTRY_IN  WHOIS_COUNTRY_JP  WHOIS_COUNTRY_NONE  \
0                 0                 0                 0                   1   
1                 0                 0                 0                   1   
2                 0                 0                 0                   1   
3                 0                 0                 0                   0   
4                 0                 0                 0                   0   

   WHOIS_COUNTRY_OTHER  WHOIS_COUNTRY_PA  WHOIS_COUNTRY_UK  WHOIS_COUNTRY_US  
0                    0                 0                 0                 0  
1                    0                 0                 0                 0  
2                    0                 0                 0                 0  
3                    0                 0                 0                 1  
4                    0                 0                 0                 1  

[5 rows x 31 columns]

In [62]:
website_dummy.dtypes

URL_LENGTH                     int64
NUMBER_SPECIAL_CHARACTERS      int64
DIST_REMOTE_TCP_PORT           int64
REMOTE_IPS                     int64
APP_BYTES                      int64
SOURCE_APP_BYTES               int64
REMOTE_APP_BYTES               int64
APP_PACKETS                    int64
DNS_QUERY_TIMES              float64
Type                           int64
CHARSET_ISO-8859-1             uint8
CHARSET_None                   uint8
CHARSET_UTF-8                  uint8
CHARSET_iso-8859-1             uint8
CHARSET_us-ascii               uint8
CHARSET_utf-8                  uint8
CHARSET_windows-1251           uint8
CHARSET_windows-1252           uint8
SERVER_Microsoft               uint8
SERVER_Other                   uint8
SERVER_nginx                   uint8
WHOIS_COUNTRY_CA               uint8
WHOIS_COUNTRY_ES               uint8
WHOIS_COUNTRY_GB               uint8
WHOIS_COUNTRY_IN               uint8
WHOIS_COUNTRY_JP               uint8
WHOIS_COUNTRY_NONE             uint8
W

# Challenge 6 - Modeling, Prediction, and Evaluation

We'll start off this section by splitting the data to train and test. **Name your 4 variables `X_train`, `X_test`, `y_train`, and `y_test`. Select 80% of the data for training and 20% for testing.**

In [63]:
from sklearn.model_selection import train_test_split

In [65]:
X_train, X_test, y_train, y_test = train_test_split(website_dummy.drop(columns = 'Type', axis=1),website_dummy['Type'],
                                                    test_size=0.20)

#### In this lab, we will try two different models and compare our results.

The first model we will use in this lab is logistic regression. We have previously learned about logistic regression as a classification algorithm. In the cell below, load `LogisticRegression` from scikit-learn and initialize the model.

In [66]:
from sklearn.linear_model import LogisticRegression

Next, fit the model to our training data. We have already separated our data into 4 parts. Use those in your model.

In [67]:
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)

C:\Users\ferna\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [69]:
y_pred = logmodel.predict(X_test)

finally, import `confusion_matrix` and `accuracy_score` from `sklearn.metrics` and fit our testing data. Assign the fitted data to `y_pred` and print the confusion matrix as well as the accuracy score

In [70]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

What are your thoughts on the performance of the model? Write your conclusions below.

In [71]:
accuracy_score(y_test,y_pred)

0.9297752808988764

In [73]:
reg_cm=confusion_matrix(y_test,y_pred)
print(reg_cm)

[[309   6]
 [ 19  22]]


In [74]:
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [75]:
plot_confusion_matrix(cm           = reg_cm, 
                      normalize    = False,
                      target_names = ['0', '1'],
                      title        = "Confusion Matrix")

#### Our second algorithm is is K-Nearest Neighbors. 

Though is it not required, we will fit a model using the training data and then test the performance of the model using the testing data. Start by loading `KNeighborsClassifier` from scikit-learn and then initializing and fitting the model. We'll start off with a model where k=3.

In [76]:
from sklearn.neighbors import KNeighborsClassifier

In [77]:
knn = KNeighborsClassifier(n_neighbors=3)

In [78]:
knn.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

To test your model, compute the predicted values for the testing sample and print the confusion matrix as well as the accuracy score.

In [79]:
pred = knn.predict(X_test)

In [80]:
print(confusion_matrix(y_test,pred))

[[302  13]
 [ 11  30]]


In [81]:
accuracy_score(y_test,pred)

0.9325842696629213

#### We'll create another K-Nearest Neighbors model with k=5. 

Initialize and fit the model below and print the confusion matrix and the accuracy score.

In [82]:
knn5 = KNeighborsClassifier(n_neighbors=5)

In [83]:
knn5.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [84]:
pred5 = knn5.predict(X_test)

In [85]:
print(confusion_matrix(y_test,pred5))

[[304  11]
 [ 13  28]]


In [86]:
accuracy_score(y_test,pred5)

0.9325842696629213

Did you see an improvement in the confusion matrix when increasing k to 5? Did you see an improvement in the accuracy score? Write your conclusions below.

In [8]:
# Quase nenhuma alteração na matriz de confusão; o accuracy score permaneceu igual.

# Bonus Challenge - Feature Scaling

Problem-solving in machine learning is iterative. You can improve your model prediction with various techniques (there is a sweetspot for the time you spend and the improvement you receive though). Now you've completed only one iteration of ML analysis. There are more iterations you can conduct to make improvements. In order to be able to do that, you will need deeper knowledge in statistics and master more data analysis techniques. In this bootcamp, we don't have time to achieve that advanced goal. But you will make constant efforts after the bootcamp to eventually get there.

However, now we do want you to learn one of the advanced techniques which is called *feature scaling*. The idea of feature scaling is to standardize/normalize the range of independent variables or features of the data. This can make the outliers more apparent so that you can remove them. This step needs to happen during Challenge 6 after you split the training and test data because you don't want to split the data again which makes it impossible to compare your results with and without feature scaling. For general concepts about feature scaling, click [here](https://en.wikipedia.org/wiki/Feature_scaling). To read deeper, click [here](https://medium.com/greyatom/why-how-and-when-to-scale-your-features-4b30ab09db5e).

In the next cell, attempt to improve your model prediction accuracy by means of feature scaling. A library you can utilize is `sklearn.preprocessing.RobustScaler` ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html)). You'll use the `RobustScaler` to fit and transform your `X_train`, then transform `X_test`. You will use logistic regression to fit and predict your transformed data and obtain the accuracy score in the same way. Compare the accuracy score with your normalized data with the previous accuracy data. Is there an improvement?

In [ ]:
# Your code here